In [30]:
import numpy as np
import pandas as pd
import polars as pl

# See readme for how I obtained the the below data
#catalog = pd.read_csv('../data/1680058493237O-result.csv', sep=',', header=0, dtype=np.float64)

# smaller sampe of the above data for testing purposes
catalog = pd.read_csv('../data/test.csv', sep=',', header=0, dtype=np.float64)

# arbitrary threashold 'k is for Kalle'
threshold_k = np.pi / 5

# helper functio
deg2rad = lambda rad: rad * np.pi / 180

# sort the catalog by magnitudes
stars = catalog.sort_values(by='b1mag')

stars

stars['dec']

0    -16.716001
1    -52.695661
2    -60.835273
3     19.182450
4     84.501680
5     72.756157
6     72.817919
7     67.499984
8     67.504577
9     65.282757
10    85.237741
11    73.194722
12    73.264174
Name: dec, dtype: float64

In [31]:
# This bit of math is a common operation in astronomy and is used to convert the spherical coordinates (r, \theta, \phi) to cartesian coordinates (x, y, z)
# To be more specific, if (RA, Dec) represents the spherical coordinates of a celestial object, then we can convert them to Cartesian coordinates (x, y, z) 
# using the following formulas:
#   x = cos(dec) * cos(ra)
#   y = cos(dec) * sin(ra)
#   z = sin(dec)
#
rays = np.stack(
            (   np.cos(deg2rad(stars['dec'])) * np.cos(deg2rad(stars['ra'])), 
                np.cos(deg2rad(stars['dec'])) * np.sin(deg2rad(stars['ra'])), 
                np.sin(deg2rad(stars['dec']))
            ) 
        ).T
rays

# Conversely, if we have the Cartesian coordinates (x, y, z), we can convert them to spherical coordinates (r, θ, φ) using the following formulas:
#
# r = sqrt(x^2 + y^2 + z^2)
# θ = arccos(z/r)
# φ = atan2(y, x)


array([[-0.18745319,  0.93921852, -0.287628  ],
       [-0.06322262,  0.60274195, -0.79542759],
       [-0.37385775, -0.31259124, -0.87322225],
       [-0.78378669, -0.52698704,  0.32857736],
       [-0.04767171,  0.08311572,  0.99539901],
       [-0.11978572,  0.27115942,  0.95505181],
       [-0.2945663 ,  0.02230101,  0.9553708 ],
       [-0.05242118,  0.37907628,  0.92387943],
       [-0.24657199, -0.29256175,  0.9239101 ],
       [ 0.05017215, -0.41511951,  0.90838238],
       [ 0.06515065, -0.05145824,  0.99654776],
       [ 0.12900926,  0.25874114,  0.95729287],
       [-0.04268434, -0.28477824,  0.95764263]])

Now we have and array of tuples with precalulated data for each star in the catalog.  Magnitude is ignored. 

In [32]:
numrows = rays.shape[0]

r1 = rays[0,:]  # get the row at position 0 of the rays collection
r2 = rays[0+1:,:] # gets the next row of the rays collection  

rr = rays[0,:] * rays[0+1:,:] # gets the next row of the rays collection  
display(rr)

sum = (rays[0,:] * rays[0+1:,:]).sum(-1) # gets the next row of the rays collection  
display("sum is ")
display(sum)

rays[0+1:,:] # this is where I loose track?  
d = np.arccos((rays[0,:] * rays[0+1:,:]).sum(-1) )
#display(d)
flat = np.flatnonzero(d < threshold_k)
display(flat)


array([[ 0.01185128,  0.5661064 ,  0.22878725],
       [ 0.07008083, -0.29359148,  0.25116317],
       [ 0.14692331, -0.49495599, -0.09450805],
       [ 0.00893621,  0.07806382, -0.28630463],
       [ 0.02245421,  0.25467795, -0.27469964],
       [ 0.05521739,  0.02094552, -0.27479139],
       [ 0.00982652,  0.35603546, -0.26573359],
       [ 0.04622071, -0.27477942, -0.26574241],
       [-0.00940493, -0.38988793, -0.26127621],
       [-0.0122127 , -0.04833053, -0.28663504],
       [-0.0241832 ,  0.24301447, -0.27534423],
       [ 0.00800131, -0.267469  , -0.27544483]])

'sum is '

array([ 0.80674493,  0.02765252, -0.44254072, -0.19930459,  0.00243252,
       -0.19862848,  0.10012839, -0.49430113, -0.66056907, -0.34717827,
       -0.05651296, -0.53491252])

array([], dtype=int64)

In [28]:
i_, j_, d_ = [],[],[]
for i in range(0, numrows):
    d = np.arccos((rays[i,:] * rays[i+1:,:]).sum(-1) )
    for j in np.flatnonzero(d < threshold_k):
        i_.append(i)
        j_.append(i+1+j)
        d_.append(d[j])

pairs = pd.DataFrame({'i':np.array(i_), 'j':np.array(j_), 'd':np.array(d_)}) #.sort_values(by='d')

pairs


,i,j,d
0,0,1,0.632174
1,4,5,0.205762
2,4,6,0.258122
3,4,7,0.305705
4,4,8,0.434461
5,4,9,0.521028
6,4,10,0.175841
7,4,11,0.252689
8,4,12,0.372001
9,5,6,0.305288
